In [ ]:
using AcausalNets
using QI
using LightGraphs
using LinearAlgebra

In [ ]:
roA = Diagonal([1/3, 1/3, 1/3])

roB = Diagonal([1/3, 1/3, 1/3])
roCwAB = Diagonal([
        0,1/2, 1/2, #A=0, B=0
        0,0,1, #A=0, B=1
        #CHANGED to that ro(C | AB) != ro(C | BA) - Originally BE 0,1,0
        0,1,0, #A=0, B=2 
        0,0,1, #A=1, B=0
        1/2,0,1/2, #A=1, B=1
        1,0,0, #A=1, B=2
        0,1,0, #A=2, B=0
        1,0,0, #A=2, B=1|
        1/2,1/2,0 #A=2, B=2
        ]); #

var_a = Variable(:a, 3)

var_b = Variable(:b, 3)
var_c = Variable(:c, 3)

sys_a = DiscreteQuantumSystem([var_a], roA)

sys_b = DiscreteQuantumSystem([var_b], roB)

sys_c_ab = DiscreteQuantumSystem([var_a, var_b], [var_c], roCwAB)
classic_an = AcausalNet()

push!(classic_an, sys_a)
push!(classic_an, sys_b)
push!(classic_an, sys_c_ab)
# show(an)

In [ ]:
roAB =1/3*(ket(0,9)+ket(4,9)+ket(8,9))* (bra(0,9)+bra(4,9)+bra(8,9)) 

roC = Diagonal([
        0,1/2,1/2, #A=0, B=0
        0,0,1, #A=0, B=1
        0,1,0, #A=0, B=2
        0,0,1, #A=1, B=0
        1/2,0,1/2, #A=1, B=1
        1,0,0, #A=1, B=2
        0,1,0, #A=2, B=0
        1,0,0, #A=2, B=1
        1/2,1/2,0 #A=2, B=2
        ]); #

var_a = Variable(:a, 3)

var_b = Variable(:b, 3)
var_c = Variable(:c, 3)

sys_ab = DiscreteQuantumSystem([var_a, var_b], roAB)
sys_c_ab = DiscreteQuantumSystem([var_a, var_b], [var_c], roCwAB)
quantum_an = AcausalNet()

push!(quantum_an, sys_ab)
push!(quantum_an, sys_c_ab)

# gplot(Graph(quantum_bn.dag), nodelabel=nodes(quantum_bn))

In [ ]:
roA = Diagonal([.5, .5])
roBwA = Diagonal([.5, .5, .4, .6])
roCwA = Diagonal([.7, .3, .2, .8])
roDwB = Diagonal([.9, .1, .5, .5])
roEwC = Diagonal([.3, .7, .6, .4])
roFwDE = Diagonal([.01, .99, .01, .99, .01, .99, .99, .01])
roGwC = Diagonal([.8, .2, .1, .9])
roHwEG = Diagonal([.05, .95, .95, .05, .95, .05, .95, .05])

var_a = Variable(:a, 2)
var_b = Variable(:b, 2)
var_c = Variable(:c, 2)
var_d = Variable(:d, 2)
var_e = Variable(:e, 2)
var_f = Variable(:f, 2)
var_g = Variable(:g, 2)
var_h = Variable(:h, 2)

sys_a = DiscreteQuantumSystem([var_a], roA)
sys_b = DiscreteQuantumSystem([var_a], [var_b], roBwA)
sys_c = DiscreteQuantumSystem([var_a], [var_c], roCwA)
sys_d = DiscreteQuantumSystem([var_b], [var_d], roDwB)
sys_e = DiscreteQuantumSystem([var_c], [var_e], roEwC)
sys_f = DiscreteQuantumSystem([var_d, var_e], [var_f], roFwDE)
sys_g = DiscreteQuantumSystem([var_c], [var_g], roGwC)
sys_h = DiscreteQuantumSystem([var_e, var_g], [var_h], roHwEG)


example_an = AcausalNet()
push!(example_an, sys_a)
push!(example_an, sys_b)
push!(example_an, sys_c)
push!(example_an, sys_d)
push!(example_an, sys_e)
push!(example_an, sys_f)
push!(example_an, sys_h)

# gplot(example_bn.dag, nodelabel=nodes(example_bn))
# [v.target for v in example_bn.cpds]


In [ ]:
# roA = Diagonal([.5, .5])

# roBwA = Diagonal([.5, .5, .5, .5])

# roCwB = Diagonal([.5, .5, .5, .5])

# roDwC = Diagonal([.5, .5, .5, .5])
# |
# sys_a = DiscreteQuantumSystem([var_a], roA)
# sys_b = DiscreteQuantumSystem([var_a], [var_b], roBwA)
# sys_c = DiscreteQuantumSystem([var_b], [var_c], roCwB)
# sys_d = DiscreteQuantumSystem([var_c], [var_d], roDwC)

# snake_an = AcausalNet()
# push!(snake_an, sys_a)
# push!(snake_an, sys_b)
# push!(snake_an, sys_c)
# push!(snake_an, sys_d)

# # gplot(snake_an.dag, nodelabel=nodes(snake_bn))

In [ ]:
import AcausalNets.Structures: DiscreteBayesNet

In [ ]:
function parent_systems(ds::S, dbn::DiscreteBayesNet) where S
    result = S[]
    for p in parents(ds)
        parent_system = systems(dbn)[variable_to_node(p, dbn)]
        if !(parent_system in result)
            push!(result, parent_system)
        end
    end
    return result
end


In [1]:
const MoralGraph = Graph

function moral_graph(dbn::DiscreteBayesNet)::MoralGraph
    result = MoralGraph(deepcopy(dbn.dag))
    for sys in systems(dbn)
        for p1 in parent_systems(sys, dbn)
            for p2 in parent_systems(sys, dbn)
                p1_ind = system_to_node(p1, dbn)
                p2_ind = system_to_node(p2, dbn)
                if(p1_ind!=p2_ind)
                    add_edge!(result, p1_ind, p2_ind)
                end
            end
        end
    end
    return result
end


LoadError: UndefVarError: Graph not defined

In [ ]:
an = example_an
moral_an = moral_graph(an)
print(ne(moral_an))
# gplot(moral_bn, nodelabel=nodes(bn))
println(moral_an.fadjlist)
println(variables_names(an))

In [ ]:
import AcausalNets.Common: is_subset

In [ ]:
const TriangulatedGraph = Graph

function triangulate(mg::MoralGraph, dbn::DiscreteBayesNet{S})::Tuple{TriangulatedGraph, Vector{Vector{S}}} where S
    mg_copy = [false for _ in vertices(mg)]
    mg = deepcopy(mg)
    nl = systems(dbn)
    cliques = Set[]
    while(!all(mg_copy))
        least_edges_to_be_added = Inf
        chosen_vertex = 0
        chosen_clique = Set()
        for v=1:length(mg_copy)
            if mg_copy[v]
                continue
            else
                clique = Set()
                for e in edges(mg)
                    if (v==src(e) || v==dst(e)) && !mg_copy[src(e)] && !mg_copy[dst(e)]
                        push!(clique, src(e))
                        push!(clique, dst(e))
                    end
                end
                edges_todo = 0
                for v1 in clique
                    for v2 in clique
                        if v1 != v2 && !in((v1, v2), edges(mg))
                            edges_todo +=1
                        end
                    end
                end
                edges_todo /= 2

                if edges_todo < least_edges_to_be_added ||
                    ((edges_todo == least_edges_to_be_added) &&
                        (prod([ncategories(systems(dbn)[n]) for n in clique]) <= prod([ncategories(systems(dbn)[n]) for n in chosen_clique])))
                    least_edges_to_be_added = edges_todo
                    chosen_vertex = v
                    chosen_clique = clique
                end
            end
        end
        chosen_nodes = Set([nl[n] for n in chosen_clique])
        if !any([is_subset(chosen_nodes, clique) for clique in cliques])
            push!(cliques, chosen_nodes)
        end
        mg_copy[chosen_vertex] = true
        for v1 in chosen_clique
            for v2 in chosen_clique
                if v1 != v2 && !in((v1, v2), edges(mg))
                    add_edge!(mg, v1, v2)
                end
            end
        end
    end
    cliques = [sort([c for c in clique], by=c -> system_to_node(c, dbn)) for clique in cliques]
    return mg, cliques
end

In [ ]:
println(ne(moral_an))

tri_moral_an, cliques = triangulate(moral_an,an)
println(ne(tri_moral_an))

# before, cliques were sets
# they are now transformed to vectors and their ordering is the same 
# as in their bayes_net DAG
# println(cliques)
# gplot(tri_moral_an ,nodelabel=nodes(an))

println(tri_moral_an.fadjlist)
println(variables_names(an))

In [ ]:
import AcausalNets.Systems:
    DiscreteSystem

In [ ]:
struct JoinTree{S <: DiscreteSystem}
    graph::Graph
    vertex_to_cluster::Dict{Int64, S}
    edge_to_sepset::Dict{Set{Int}, S}
end


In [ ]:
function sepset_cost(sepset::Set{DiscreteQuantumSystem}, an::AcausalNet)
    weights = [ncategories(s) for s in sepset]
    
    return 0
end

In [ ]:
function sepset_comparator(c1::Vector{S}, c2::Vector{S})::Int64 where S
    -length(intersect(c1, c2))
end

In [ ]:
function clique_size(clique::Vector{S})::Int64 where S
    prod([ncategories(s) for s in clique])
end


In [ ]:
family(ds::S, dbn::DiscreteBayesNet{S}) where {S} = union(Set([ds]), Set(parent_systems(ds, dbn)))

In [ ]:
function parent_cliques_dict(cliques::Vector{Vector{S}}, dbn::DiscreteBayesNet{S})::Dict{S, Vector{S}} where S
    Dict([
        sys => first([
            c for c in cliques if is_subset(family(sys, dbn), Set(c))
            ])
        for sys in systems(dbn)
    ])
end

In [ ]:
pr_cli_dict = parent_cliques_dict(cliques, an)

for sys in keys(pr_cli_dict)
    println(variables_names(sys), " => ", [variables_names(s) for s in pr_cli_dict[sys]])
end

In [ ]:
import AcausalNets.Systems: identity_distribution

In [ ]:
Diagonal <: AbstractArray{T,2} where T

In [ ]:
distribution(ds::DiscreteSystem) = ds.distribution

In [ ]:
import AcausalNets.Systems: QuantumDistribution
import AcausalNets.Algebra: star, unstar

In [ ]:
multiply_star(d1::QuantumDistribution, d2::QuantumDistribution) = star(d1, d2)

In [ ]:
divide_star(d1::QuantumDistribution, d2::QuantumDistribution) = unstar(d1, d2)

In [ ]:
multiply_kron(d1::QuantumDistribution, d2::QuantumDistribution) = kron(d1, d2)

In [ ]:
function permute_distribution(d::QuantumDistribution, dimensions::Vector{Int64}, order::Vector{Int64})
    permute_systems(d, dimensions, order)
end

In [ ]:
function reduce_distribution(d::QuantumDistribution, dimensions::Vector{Int64}, reduce_ind::Vector{Int64})
    if length(reduce_ind) > 0 
        ptrace(d, dimensions, reduce_ind)
    else
        d
    end
end

In [ ]:
Int64[x for x in [1,2,4]]

In [ ]:
invperm(vcat(Int64[1, 2, 4, 3], Int64[]))

In [ ]:
function merge_systems(systems::Vector{S})::S where {D, S<:DiscreteSystem{D}}
    all_variables = Variable[]
    all_parents = Variable[]
    for s in systems
        for v in variables(s)
            !(v in all_variables) || error("$v duplicated!")
            push!(all_variables, v)
        end
    end
    for s in systems
        for p in parents(s)
            if !(p in all_variables) && (p in all_parents)
                push!(all_parents, p)
            end
        end
    end
    all_relevant_variables = vcat(all_parents, all_variables)
    target_size = prod([ncategories(v) for v in all_relevant_variables])
    result_distribution = identity_distribution(D, target_size)
    
    for sys in systems
        sys_distribution = distribution(sys)
        sys_variables = relevant_variables(sys)
        sys_indices = Int64[
            findfirst([v == var for var in all_relevant_variables]) 
            for v in sys_variables
            ]
        sys_dimensions = [ncategories(v) for v in sys_variables]
        
        other_variables = [v for v in all_relevant_variables if !(v in sys_variables)]
        other_indices = Int64[
            findfirst([v == var for var in all_relevant_variables]) 
            for v in other_variables
            ]
        other_dimensions = [ncategories(v) for v in other_variables]
        other_distribution = identity_distribution(D, prod(other_dimensions))
        
        factor_distribution = multiply_kron(sys_distribution, other_distribution)        
        factor_indices = vcat(sys_indices, other_indices)
        factor_dimensions = vcat(sys_dimensions, other_dimensions)
        
#         println(sys_variables)
#         println("sys $sys_indices")
#         println("other $other_indices")
#         println("factor $factor_indices")
#         println("dim_sys $(size(sys_distribution))")        
#         println("dim_other $(size(other_distribution))")
#         println()
        right_order = invperm(factor_indices)
        ordered_distribution = permute_distribution(factor_distribution, factor_dimensions, right_order)
        result_distribution = multiply_star(ordered_distribution, result_distribution)
    end
#     println("-------")
    DiscreteSystem{D}(all_parents, all_variables, result_distribution)
end

In [ ]:
sys_abc = merge_systems([sys_a, sys_b, sys_c])
d_abc = distribution(sys_abc)

In [ ]:
ptrace(d_abc, [2,2,2], [3])

In [ ]:
function Cluster(systems::Vector{S})::S where S
    all_variables = Variable[]
    all_parents = Variable[]
    for s in systems
        for v in variables(s)
            !(v in all_variables) || error("$v duplicated!")
            push!(all_variables, v)
        end
    end  
    for s in systems
        all([p in all_variables for p in parents(s)]) || 
            error("parents of system $variables(sys) outside cluster!")
    end
    merge_systems(systems)
end

In [ ]:
function identity_system(system::DiscreteSystem{D})::DiscreteSystem{D} where D
    return DiscreteSystem{D}(variables(system), identity_distribution(D, ncategories(system)))
end

In [ ]:
function identity_system(::Type{DiscreteSystem{D}}, variables::Vector{Variable})::DiscreteSystem{D} where D
    target_size = prod([ncategories(v) for v in variables])
    DiscreteSystem{D}(variables, identity_distribution(D, target_size))
end

In [ ]:
identity_system(DiscreteQuantumSystem, variables(sys_b)).distribution

In [ ]:
function sys_or_id(system::S, clique::Vector{S}, parent_cliques::Dict{S, Vector{S}})::S where S
    if parent_cliques[system] == clique
        return system
    else
        return identity_system(system)
    end
end

In [ ]:
function JoinTree(cliques::Vector{Vector{S}}, dbn::DiscreteBayesNet{S})::JoinTree{S} where S
    candidate_sepsets = []
    trees = Dict([c => c for c in cliques])
    chosen_sepsets = Set()

    parent_cliques = parent_cliques_dict(cliques, dbn)
    jt = JoinTree{S}(
        Graph(length(cliques)),
        Dict([
            i => Cluster([sys_or_id(sys, cliques[i], parent_cliques) for sys in cliques[i]])
            for i in 1:length(cliques)
            ]),
        Dict()
    )

    for c1 = 1:length(cliques)
        for c2= 1:length(cliques)
            if c1 != c2
                push!(candidate_sepsets,(c1, c2))
            end
        end
    end

    candidate_sepsets = sort(candidate_sepsets, by=c -> sepset_comparator(cliques[c[1]], cliques[c[2]]))
    i = 1
    n = length(cliques)

    while length(chosen_sepsets) < n-1
        i1, i2 = candidate_sepsets[i]
        c1, c2 = cliques[i1], cliques[i2]
        sepset = intersect(c1, c2)
        if (trees[c1] != trees[c2]) && !any([sepset==s for s in chosen_sepsets])
            push!(chosen_sepsets, sepset)
            trees[c1] = trees[c2] = union(c1, c2)
            add_edge!(jt.graph, i1, i2)
            sepset_variables = reduce(vcat, [variables(s) for s in sepset])
            push!(jt.edge_to_sepset, Set([i1, i2]) => identity_system(S, sepset_variables))
        end
        i += 1
    end
    jt
end

In [ ]:
jt = JoinTree(cliques, an)

jt.vertex_to_cluster[1].variables

In [ ]:
event(system::AbstractMatrix, e::AbstractMatrix) = (e * system * e) / tr(e * system)

In [ ]:
[(c, variables(jt.vertex_to_cluster[c])) for c in keys(jt.vertex_to_cluster)]

In [ ]:
shallowcopy(ds::DiscreteSystem{D}) where D = DiscreteSystem{D}(
    parents(ds),
    variables(ds),
    deepcopy(distribution(ds))
    ) 

In [ ]:
shallowcopy(jt::JoinTree) = JoinTree(
        jt.graph, 
        Dict([
            k => shallowcopy(jt.vertex_to_cluster[k]) for k in keys(jt.vertex_to_cluster)
            ]),
        Dict([
            k => shallowcopy(jt.edge_to_sepset[k]) for k in keys(jt.edge_to_sepset)
            ])
    )

In [ ]:
shallowcopy(jt)

In [ ]:
import AcausalNets.Algebra: eye

In [ ]:
# from this it would seem that currently node [a, d, c] 
# holds conditional probability ADCwD

a_factor = kron(kron(roA, eye(ncategories(sys_d))), eye(ncategories(sys_c)))
c_factor = kron(eye(ncategories(sys_d)), roCwA)
c_factor = permute_systems(c_factor, [2,2,2], [2,1,3])
ADCwD = star(c_factor, a_factor)
tr(ADCwD)
ptrace(ADCwD, [2,2,2], [1,2])

In [ ]:
roBwA

In [ ]:
# [a, b, d] holds ABDwA
b_factor = kron(roBwA, eye(ncategories(sys_d)))
d_factor = kron(eye(ncategories(sys_a)), roDwB)

ABDwA = star(b_factor, d_factor)
# trace(ABDwA)
ptrace(ABDwA, [2,2,2], [3])

In [ ]:
[(c, variables(jt.vertex_to_cluster[c])) for c in keys(jt.vertex_to_cluster)]

In [ ]:
for (k,v) in parent_cliques_dict(cliques, an)
    println(variables(k))
    println([variables(c) for c in v])
    println()
end

This is a raw message pass

In [ ]:
abc = jt.vertex_to_cluster[6]
ptrace(distribution(abc), [2,2,2], [2,3])

In [ ]:
BCd = jt.vertex_to_cluster[5]
ptrace(distribution(BCd), [2,2,2], [2,3])

In [ ]:
# message from abc to BCd
bc = ptrace(distribution(abc), [2,2,2], [1])

In [ ]:
# fitting the message into BCd
bcD = kron(bc, eye(ncategories(sys_d)))

In [ ]:
bcd = star(bcD, distribution(BCd))
# ptrace(ABD, [2,2,2], [2,3])
tr(bcd)
ptrace(bcd, [2,2,2], [1,2])

In [ ]:
function single_message_pass(from_ind::Int, to_ind::Int, jt::JoinTree{S}) where S
    if (from_ind, to_ind) in edges(jt.graph)
        jt = shallowcopy(jt)
        cluster_from = jt.vertex_to_cluster[from_ind]
        cluster_to = jt.vertex_to_cluster[to_ind]
        edge_set = Set([from_ind, to_ind])
        sepset = jt.edge_to_sepset[edge_set]
#         println("message from $(variables(cluster_from)), to $(variables(cluster_to))")
#         println("through $(variables(sepset))")
#         println(cluster_from, " ", sepset, " ", cluster_to)
        to_trace_out_vars = setdiff(variables(cluster_from), variables(sepset))
        to_trace_out_ind = Int64[
            findfirst([v==var for var in variables(cluster_from)]) for v in to_trace_out_vars
            ]
#         println("trace_out ", to_trace_out_ind)
        from_variables_sizes = [ncategories(v) for v in variables(cluster_from)]
#         println(from_variables_sizes)
        from_distribution = distribution(cluster_from)
        old_sepset_distribution = distribution(sepset)
        new_sepset_distribution = reduce_distribution(
            distribution(cluster_from), from_variables_sizes, to_trace_out_ind
        )
#         println(new_sepset_num)
#         println(old_sepset_num)
        
#         println(jt.edge_to_num[Set([from_ind, to_ind])])
        sepset = S(variables(sepset), new_sepset_distribution)
        jt.edge_to_sepset[edge_set] = sepset
#         println(jt.edge_to_num[Set([from_ind, to_ind])])
        
        to_distribution = distribution(cluster_to)
        
#         message = event(old_sepset_num, new_sepset_num) 
        message = divide_star(new_sepset_distribution, old_sepset_distribution)
        
        message_vars = variables(sepset)
        non_message_vars = setdiff(variables(cluster_to), message_vars)
        non_message_size = prod([ncategories(v) for v in non_message_vars])     
#         println(message_sym)
        message = multiply_kron(
            message, identity_distribution(typeof(message), non_message_size)
        )
        message_all_vars = vcat(message_vars, non_message_vars)
#         println(message_sym)        println("all_vars $message_all_vars")
#         println("message_vars $message_vars")
#         println("non_message_vars $non_message_vars")
#         println("all_vars $message_all_vars")
        message_sorted_indices = Int64[
            findfirst([v==var for var in variables(cluster_to)]) 
            for v in message_all_vars
            ]
                                       
#         println(message_sorted_inds) 
        message_dims = [ncategories(v) for v in message_all_vars]
        message_ordered = permute_distribution(message, message_dims, message_sorted_indices)
        cluster_to = S(variables(cluster_to), multiply_star(to_distribution, message_ordered))
        jt.vertex_to_cluster[to_ind] = cluster_to
    end
#     println()
    return jt
end

In [ ]:
single_message_pass(6, 5, jt)

In [ ]:
[(k, variables(v)) for (k,v) in jt.vertex_to_cluster]

In [ ]:
init_jt.vertex_to_num[5]
#bcd
ptrace(jt.distributionvertex_to_cluster[5], [2,2,2], [2,3])

In [ ]:
init_jt.vertex_to_num[6]
#abc
ptrace(init_jt.vertex_to_num[6], [2,2,2], [2,3])

In [ ]:
# bcd -> abc
passed = single_message_pass(5, 6, init_jt, an);

In [ ]:
passed.vertex_to_num[5]

In [ ]:
passed.vertex_to_num[6]

In [ ]:
ptrace(passed.vertex_to_num[5], [2,2,2], [2, 3])

In [ ]:
# abc -> bcd 
passed_2 = single_message_pass(6, 5, passed, an);
passed_2.vertex_to_num[5]
ptrace(passed_2.vertex_to_num[5], [2,2,2], [1, 3])

In [ ]:
function collect_evidence(cluster_ind::Int, cluster_marks::Vector{Bool}, jt::JoinTree)
    jt = shallowcopy(jt)
    cluster_marks[cluster_ind] = false
    for neighbor in neighbors(jt.graph, cluster_ind)
        if cluster_marks[neighbor]
#             println(jt.clusters[cluster_ind], " ", jt.clusters[neighbor])
            jt, cluster_marks = collect_evidence(neighbor, cluster_marks, jt)
            jt = single_message_pass(neighbor, cluster_ind, jt)
            # pass a message from neighbor to cluster_ind
        end
        
    end
    jt, cluster_marks
    
end

In [ ]:
function distribute_evidence(cluster_ind::Int, cluster_marks::Vector{Bool}, jt::JoinTree)
    cluster_marks[cluster_ind] = false
    for neighbor in neighbors(jt.graph, cluster_ind)
        if cluster_marks[neighbor]
            jt = single_message_pass(cluster_ind, neighbor, jt)
            # pass a message from cluster_ind to neighbor
        end
    end
    for neighbor in neighbors(jt.graph, cluster_ind)
        if cluster_marks[neighbor]
            jt, cluster_mars = distribute_evidence(neighbor, cluster_marks, jt)
        end
    end
    jt = shallowcopy(jt)
    jt, cluster_marks
end

In [ ]:
function global_propagation(jt::JoinTree)
    jt = shallowcopy(jt)
    cluster_marks = [true for k in keys(jt.vertex_to_cluster)]
    arbitrary_cluster_ind = 1
#     println("Collect evidence")
    jt, cluster_marks = collect_evidence(arbitrary_cluster_ind, cluster_marks, jt)
    cluster_marks = [true for k in keys(jt.vertex_to_cluster)]
#     println("Distribute evidence")
    jt, cluster_marks = distribute_evidence(arbitrary_cluster_ind, cluster_marks, jt)
    return jt
end

In [ ]:
propagated_jt = global_propagation(jt)
# propagated_jt = global_propagation(propagated_jt, bn)

In [ ]:
[variables(c) for c in propagated_jt.clusters]

In [ ]:
BCD = propagated_jt.vertex_to_num[5]

In [ ]:
ptrace(BCD, [2,2,2], [1,3])

In [ ]:
ABC = propagated_jt.vertex_to_num[6]

In [ ]:
ptrace(ABD, [2,2,2], [1,2])

In [ ]:
# CDE
ptrace(propagated_jt.vertex_to_num[4], [2,2,2], [2,3])


# To sum everything up

In [ ]:
an = example_an
# gplot(bn.dag, nodelabel=nodes(bn))

In [ ]:
moral_g = moral_graph(an)
tri_moral_graph, cliques = triangulate(moral_g, an)
jt = JoinTree(cliques, an);

In [ ]:
a = 0
a_oper = ketbra(a,a,2)
assignments = Dict([
        var_a => a_oper
        ])

In [ ]:
# initialized_jt = initialize(jt, an, assignments)
propagated_jt = global_propagation(jt)
[(k, variables(v)) for (k,v) in propagated_jt.vertex_to_cluster]

In [ ]:
cde = distribution(propagated_jt.vertex_to_cluster[4])
ptrace(cde, [2,2,2], [2])

In [ ]:
ceg = distribution(propagated_jt.vertex_to_cluster[2])
ptrace(ceg, [2,2,2], [3])